### **First, Roll Back Colab to the right dependencies for SparseML**

In [ ]:
# !pip uninstall -y torch torchvision torchtext torchaudio fastai flask

In [ ]:
# !pip install "sparseml[torch, torchvision]"

### **Note: must restart the runtime after this step**

In [1]:
import sparseml
import sparsezoo
import torch
import torchvision

In [2]:
import pkg_resources

pkg_resources.get_distribution("torch")

torch 1.9.1 (/usr/local/lib/python3.7/dist-packages)

In [3]:
from sparseml.pytorch.models import ModelRegistry
from sparseml.pytorch.datasets import ImagenetteDataset, ImagenetteSize
from sparsezoo import Model

## 1. Load in pre-sparsified ResNet50 model from Model Zoo

In [4]:
zoo_stub_path = (
    "zoo:cv/classification/resnet_v1-50/pytorch/sparseml/imagenet/pruned_quant-moderate"
    "?recipe=transfer_learn"
)

In [5]:
model = ModelRegistry.create(
    key="resnet50",
    pretrained_path=zoo_stub_path,
    pretrained_dataset="imagenette",
    num_classes=10,
    ignore_error_tensors=["classifier.fc.weight", "classifier.fc.bias"],
)

input_shape = ModelRegistry.input_shape("resnet50")
input_size = input_shape[-1]

print(model)

downloading...:   0%|          | 0.00/195M [00:00<?, ?B/s]

ResNet(
  (input): _Input(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (sections): Sequential(
    (0): Sequential(
      (0): _BottleneckBlock(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running

## 2. Load in datasets

We can see that the transforms are going to be applied...

In [6]:
train_data = ImagenetteDataset(
    train=True, dataset_size=ImagenetteSize.s320, image_size=input_size
)

val_data = ImagenetteDataset(
    train=False, dataset_size=ImagenetteSize.s320, image_size=input_size
)

downloading imagenette ImagenetteSize.s320:   0%|          | 0.00/325M [00:00<?, ?B/s]

already downloaded imagenette ImagenetteSize.s320


In [7]:
print(train_data)
print(val_data)

Dataset ImagenetteDataset
    Number of datapoints: 12894
    Root location: /root/.cache/nm_datasets/imagenette/imagenette-320/train
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImagenetteDataset
    Number of datapoints: 500
    Root location: /root/.cache/nm_datasets/imagenette/imagenette-320/val
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


## 3. Create training loop and objects for the transfer learning

In [8]:
from tqdm.auto import tqdm
import math
import torch

In [9]:
def run_one_epoch(model, loader, criterion, device, train=False, optimizer=None):
    if train:
        model.train()
    else:
        model.eval()
        
    running_loss = 0.0
    total_correct = 0
    total_predictions = 0
    
    for step, (inputs, labels) in tqdm(enumerate(loader), total=len(loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        if train:
            optimizer.zero_grad()
        
        outputs, _ = model(inputs) # logits and softmax as tuble is returned
        loss = criterion(outputs, labels)
        
        if train:
            loss.backward()
            optimizer.step()
            
        running_loss += loss.item()
        predictions = outputs.argmax(dim=1)
        total_correct += torch.sum(predictions == labels).item()
        total_predictions += inputs.shape[0]
        
    loss = running_loss / (step + 1.0)
    accuracy = total_correct / total_predictions
    return loss, accuracy

In [10]:
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

In [11]:
BATCH_SIZE = 32
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f'Using device: {device}')

train_loader = DataLoader(train_data, BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, BATCH_SIZE, shuffle=False, num_workers=4)

criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=8e-3)

Using device: cuda


In [12]:
next(iter(train_loader))[0].shape

torch.Size([32, 3, 224, 224])

In [13]:
from sparseml.pytorch.utils import get_prunable_layers, tensor_sparsity

for name,layer in get_prunable_layers(model):
    print(f"{name}.weight: {tensor_sparsity(layer.weight).item():4f}")

input.conv.weight: 0.000000
sections.0.0.conv1.weight: 0.700195
sections.0.0.conv2.weight: 0.699978
sections.0.0.conv3.weight: 0.699951
sections.0.0.identity.conv.weight: 0.699951
sections.0.1.conv1.weight: 0.699951
sections.0.1.conv2.weight: 0.750000
sections.0.1.conv3.weight: 0.699951
sections.0.2.conv1.weight: 0.699951
sections.0.2.conv2.weight: 0.750000
sections.0.2.conv3.weight: 0.699951
sections.1.0.conv1.weight: 0.700073
sections.1.0.conv2.weight: 0.750000
sections.1.0.conv3.weight: 0.700012
sections.1.0.identity.conv.weight: 0.750000
sections.1.1.conv1.weight: 0.750000
sections.1.1.conv2.weight: 0.750000
sections.1.1.conv3.weight: 0.750000
sections.1.2.conv1.weight: 0.750000
sections.1.2.conv2.weight: 0.750000
sections.1.2.conv3.weight: 0.700012
sections.1.3.conv1.weight: 0.700012
sections.1.3.conv2.weight: 0.750000
sections.1.3.conv3.weight: 0.750000
sections.2.0.conv1.weight: 0.699982
sections.2.0.conv2.weight: 0.799995
sections.2.0.conv3.weight: 0.750000
sections.2.0.identit

We can see the model is quite sparse

## 4. Run transfer learning

In [14]:
from sparsezoo import Model

zoo_model = Model(zoo_stub_path)
recipe_path = zoo_model.recipes.default.path
print(f"Recipe downloaded to {recipe_path}")

downloading...:   0%|          | 0.00/3.10k [00:00<?, ?B/s]

Recipe downloaded to /root/.cache/sparsezoo/9e729048-5c2a-49fb-a08d-d419cdbef853/recipe/recipe_transfer_learn.md


In [15]:
from sparseml.pytorch.optim import ScheduledModifierManager

manager = ScheduledModifierManager.from_yaml(recipe_path)
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))

In [16]:
# this is the number of stpes in the iteration
print(len(train_loader))
print(len(val_loader))

403
16


In [17]:
epochs = manager.min_epochs
for epoch in range(manager.max_epochs):
    epoch_name = f"{epoch + 1}/{manager.max_epochs}"
    
    print(f"Running Training Epoch {epoch_name}")
    t_loss, t_acc = run_one_epoch(model, train_loader, criterion, device, train=True, optimizer=optimizer)
    print(f"Training Epoch: {epoch_name}\nTraining Loss: {t_loss}\nTop 1 Acc: {t_acc}\n")
    
    print(f"Running Validation Epoch {epoch_name}")
    v_loss, v_acc = run_one_epoch(model, val_loader, criterion, device)
    print(f"Validation Epoch: {epoch_name}\nValidation Loss: {v_loss}\nTop 1 Acc: {v_acc}\n")
    
manager.finalize(model)

Running Training Epoch 1/10


  0%|          | 0/403 [00:00<?, ?it/s]

Training Epoch: 1/10
Training Loss: 1.250252147466314
Top 1 Acc: 0.6185822863347293

Running Validation Epoch 1/10


  0%|          | 0/16 [00:00<?, ?it/s]

Validation Epoch: 1/10
Validation Loss: 1.8413323909044266
Top 1 Acc: 0.486

Running Training Epoch 2/10


...

Validation Epoch: 9/10
Validation Loss: 0.2240769749332685
Top 1 Acc: 0.944

Running Training Epoch 10/10


  0%|          | 0/403 [00:00<?, ?it/s]

Training Epoch: 10/10
Training Loss: 0.022878981572223445
Top 1 Acc: 0.99457111834962

Running Validation Epoch 10/10


  0%|          | 0/16 [00:00<?, ?it/s]

Validation Epoch: 10/10
Validation Loss: 0.2087052022689022
Top 1 Acc: 0.954



In [18]:
from sparseml.pytorch.utils import get_prunable_layers, tensor_sparsity

for name,layer in get_prunable_layers(model):
    print(f"{name}.weight: {tensor_sparsity(layer.weight).item():4f}")

input.conv.module.weight: 0.000000
sections.0.0.conv1.module.weight: 0.700195
sections.0.0.conv2.module.weight: 0.699978
sections.0.0.conv3.module.weight: 0.699951
sections.0.0.identity.conv.module.weight: 0.699951
sections.0.1.conv1.module.weight: 0.699951
sections.0.1.conv2.module.weight: 0.750000
sections.0.1.conv3.module.weight: 0.699951
sections.0.2.conv1.module.weight: 0.699951
sections.0.2.conv2.module.weight: 0.750000
sections.0.2.conv3.module.weight: 0.699951
sections.1.0.conv1.module.weight: 0.700073
sections.1.0.conv2.module.weight: 0.750000
sections.1.0.conv3.module.weight: 0.700012
sections.1.0.identity.conv.module.weight: 0.750000
sections.1.1.conv1.module.weight: 0.750000
sections.1.1.conv2.module.weight: 0.750000
sections.1.1.conv3.module.weight: 0.750000
sections.1.2.conv1.module.weight: 0.750000
sections.1.2.conv2.module.weight: 0.750000
sections.1.2.conv3.module.weight: 0.700012
sections.1.3.conv1.module.weight: 0.700012
sections.1.3.conv2.module.weight: 0.750000
sec

In [40]:
import os
from sparseml.pytorch.utils import ModuleExporter

save_dir = "pytorch_sparse_quantized_transfer_learning"
quant_onnx_graph_name = "resnet50_imagenette_pruned_quant.onnx"
quantized_onnx_path = os.path.join(save_dir, quant_onnx_graph_name)

exporter = ModuleExporter(model, output_dir=save_dir)
exporter.export_pytorch(name="resnet50_imagenette_pruned_qat.pth")
exporter.export_onnx(
    torch.randn(1, 3, 224, 224), name=quant_onnx_graph_name, convert_qat=True
)

print(f"Sparse-Quantized ONNX model saved to {quantized_onnx_path}")

2022-09-07 21:09:09 sparseml.pytorch.sparsification.quantization.quantize_qat_export INFO     Converted 0 quantizable Gemm ops with weight and bias to MatMulInteger and Add
INFO:sparseml.pytorch.sparsification.quantization.quantize_qat_export:Converted 0 quantizable Gemm ops with weight and bias to MatMulInteger and Add


Sparse-Quantized ONNX model saved to pytorch_sparse_quantized_transfer_learning/resnet50_imagenette_pruned_quant.onnx


In [49]:
model = model.to('cpu')

In [50]:
model.eval()

num_correct = 0
num_evaluated = 0

for batch in tqdm(iter(val_loader)):
  x, y = batch
  logits, _ = model(x)
  y_hat = logits.argmax(dim=1)

  num_correct += torch.sum(y_hat == y).item()
  num_evaluated += y.shape[0]

print(f"num evaluated = {num_evaluated}")
print(f"num correct = {num_correct}")
print(f"accuracy = {num_correct / num_evaluated}")

  0%|          | 0/16 [00:00<?, ?it/s]

num evaluated = 500
num correct = 477
accuracy = 0.954
